# Train your first Deep Reinforcement Learning Agent

PYTHON 3.9

El objetivo de este notebook es crear un sistema para entrenar los algoritmos que sea claro y escalable. Así que voy a tratar el menor número de celdas posibles

In [ ]:
import sys #-- lo he necesitado para importar el paquete
sys.path.append('..')
import gym_hearthstone

## Argumentos
He añadido argumentos al environment para que sea más sencillo hacer el entrenamiento:

    action_type: Define el espacio de acción que se va a crear. Ahora mismo solo hay tres implementados.
    
        * "random"  -> Solo una acción possible (Tomar una acción aleatoria de las que son posibles).
        * "type_rd" -> 6 acciones posibles Realiza una acción aleatoria del tipo de acción devuelto por el DQN (en caso de tener alguna carta) y si no tengo ninguna carta del tipo de acción que ha devuelto el DQN entonces hace una acción aleatoria de entre las disponibles). NO TIENE SENTIDO PORQUE CONFUNDE AL DQN
        * "type"    -> 6 acciones posibles (Realiza una acción aleatoria del tipo de acción devuelto por el DQN (en caso de tener alguna carta) y si no tengo ninguna carta del tipo de acción que ha devuelto el DQN no hace nada. A continuación vuelve a invocar al DQN para obtener otro tipo de acción).
        
        
        
    reward_mode: Define el reward que devuelve el environment:
    
        * "simple"    -> Solo hay reward cuando se gana o se pierde.
        * "penalize"  -> Se penalizan las acciones que no son válidas.
        * "incentive" -> Se premian las acciones que son válidas.
        * "complex"   -> Se penalizan las acciones que no son válidas y se premian las válidas.


    steps: Por ahora usa el numero de steps para el nombre del fichero de salida. 
    
        

In [ ]:
import gymnasium as gym
import gym_hearthstone
from stable_baselines3 import DQN
import time
import os

In [ ]:

steps = 100
run = -1
env = gym.make('Hearthstone-v1', action_type = "random", reward_mode = "simple")

In [ ]:
run += 1
## Generar Modelo
model = DQN("MultiInputPolicy", env, verbose=1)

# Resetear env y stats
env.reset_stats()
start = time.time()
#env.reset()

## Entrenar modelo
model.learn(total_timesteps=steps, log_interval=4)
end = time.time()




In [ ]:
## Calcular estadísticas
env.display_stats()

total_time = end-start

if(env.curr_episode > 0):
    avg_time = (end-start) / env.curr_episode
    avg_steps = steps / env.curr_episode
else:
    avg_time = (end-start) 
    avg_steps = steps 


if (env.wins + env.losses > 0):
    win_rate = env.wins / (env.wins + env.losses)
else:
    win_rate = 0

print("Avg steps per game: " + str(avg_steps))
print("Total running time: {:.2f} and running time per game: {:.2f}".format(total_time, avg_time) )

## Crear ficheros
file_name = str(env.action_type) + "__" + str(env.reward_mode) + "__" + str(steps) + "__" + str(run)

# METRICS
data_file = open('./metrics/' + file_name + '.txt', 'w')
data_file.write("Games, AvgStepsGame, TotalTime, AvgTimeGame, TotalReward, WinRate \n")
data_file.write("{},{:.2f},{:.2f},{:.2f},{:.2f},{:.2f}".format(env.curr_episode, avg_steps, total_time, avg_time, env.total_reward, win_rate))
data_file.close()

# REWARD DATA
data_file = open('./data/' + file_name + '_reward.csv', 'w')
data_file.write("Games, Reward \n")
for key in env.reward_dict:
    data_file.write("{},{:.2f} \n".format(key, env.reward_dict[key]))
data_file.close()

# Train Games
data_file = open("./data/" + file_name + '_games.csv', 'w')
for l in env.games_outcome:
    data_file.write(str(l) + "\n")
data_file.close()

model.save("./dqn_models/" + file_name + "_dqn")
env.close()
